In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

%matplotlib inline
pd.set_option('max_colwidth', -1)

Randomize the headers

In [3]:
from user_agent import generate_user_agent
headers = {'User-Agent': generate_user_agent(device_type="desktop", os=('mac', 'linux'))}

In [4]:
url = 'https://saveyourinternet.eu/'

raw_html = requests.get(url, headers=headers)
soup_doc = BeautifulSoup(raw_html.content, 'html.parser')

Create a list of the html for each Member State, conveniently coded as table rows ('tr' html tags). Drop the first three rows as these entries are the overall outcome.

In [80]:
member_states_raw = soup_doc.findAll('tr')[3:]
len(member_states_raw)

28

Number of entries checks out, let's create our database now

In [105]:
council_vote_list = []

for each in member_states_raw:
    country_dict = {}
    country_dict['country'] = each.strong.string
    # print(each.strong.string)
    country_dict['percent_eu_population'] = float(each.em.string[:-18]) #converting to floats now for easier cleaning
    
    #print(float(each.em.string[:-18])) #converting to floats now for easier cleaning
    #print(each.span['class'][-1])
    
    
    if each.span['class'][-1] == 'fa-arrow-circle-up':
        country_dict['copyright_directive_vote'] = 1
        #print('supported the directive')
    elif each.span['class'][-1] == 'fa-circle-thin':
        country_dict['copyright_directive_vote'] = 0
        # print('abstained')
    elif each.span['class'][-1] == 'fa-arrow-circle-down':
        country_dict['copyright_directive_vote'] = -1
        #print('voted against')
    council_vote_list.append(country_dict)

        
        

In [106]:
council_vote_list

[{'country': 'Austria',
  'percent_eu_population': 1.71,
  'copyright_directive_vote': 1},
 {'country': 'Belgium',
  'percent_eu_population': 2.22,
  'copyright_directive_vote': 0},
 {'country': 'Bulgaria',
  'percent_eu_population': 1.37,
  'copyright_directive_vote': 1},
 {'country': 'Croatia',
  'percent_eu_population': 0.8,
  'copyright_directive_vote': 1},
 {'country': 'Cyprus',
  'percent_eu_population': 0.17,
  'copyright_directive_vote': 1},
 {'country': 'Czech Republic',
  'percent_eu_population': 2.04,
  'copyright_directive_vote': 1},
 {'country': 'Denmark',
  'percent_eu_population': 1.13,
  'copyright_directive_vote': 1},
 {'country': 'Estonia',
  'percent_eu_population': 0.26,
  'copyright_directive_vote': 0},
 {'country': 'Finland',
  'percent_eu_population': 1.07,
  'copyright_directive_vote': -1},
 {'country': 'France',
  'percent_eu_population': 13.1,
  'copyright_directive_vote': 1},
 {'country': 'Germany',
  'percent_eu_population': 16.12,
  'copyright_directive_vot

In [107]:
df = pd.DataFrame(council_vote_list)
df

,copyright_directive_vote,country,percent_eu_population
0,1,Austria,1.71
1,0,Belgium,2.22
2,1,Bulgaria,1.37
3,1,Croatia,0.80
4,1,Cyprus,0.17
5,1,Czech Republic,2.04
6,1,Denmark,1.13
7,0,Estonia,0.26
8,-1,Finland,1.07
9,1,France,13.10


In [108]:
df.to_csv('council_vote.csv', index=False)